# Metodos Numericos - Laboratorio 6 Ecuaciones Diferenciales Ordinarias

Grupo 9

Presentado por:

**Juan Diego Rozo Álvarez - Universidad Nacional de Colombia**

**Juan Pablo Gómez Cristancho - Universidad Nacional de Colombia**

---
En este laboratorio se hará uso de métodos numericos para aproximas soluciones numéricas a Ecuaciones Diferenciales Ordinarias en problemas de contornos por medio de métodos como el **Método del Disparo Lineal** y el **Método de las Diferencias Finitas**

La implementación de estos metodos será realizada en su totalidad haciendo el uso en Python.

A continuación se explica la estructura del documento en Collab.

* Definicion de librerias.
* Definicion de getTabla para imprimir los datos almacenados.
* Definicion del método del Disparo Lineal y sus derivados.
* Implementacion del método disparo_main(args) y sus derivados.
* Definicion del método de Diferencias Finitas   derivados.
* Implementacion del metodo main(args) y derivados.

In [2]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


# Definición del método del Disparo Lineal
---
El método del disparo lineal transforma un problema de contorno para una EDO de segundo orden
en un conjunto de problemas de valor inicial que se resuelven mediante integración numérica.

Dado un problema de contorno de la forma:

$$ y''(t) = f(t, y, y'), \quad y(a) = \alpha, \quad y(b) = \beta $$

Se realiza lo siguiente:

1. Se reduce la EDO de segundo orden a un sistema de primer orden, con:
$$ x = y \quad \text{y} \quad y = y' $$
   De esta forma, se obtiene el sistema:
   $$ x' = y $$
   $$ y' = f(t, x, y) $$

2. Se efectúan dos disparos:
   - Primer disparo con condición inicial $y'(a) = 0$
   - Segundo disparo con condición inicial $y'(a) = 1$

3. Se obtiene la combinación lineal de las soluciones para aproximar la solución que cumple
   con $y(b) = \beta$, interpolando linealmente con:

$$ s = \frac{\beta - y_1(b)}{y_2(b) - y_1(b)} $$

Donde $y_1(b)$ y $y_2(b)$ son las soluciones de los disparos en el punto final.
La solución final se reconstruye como:

$$ y(t) = x_1(t) + s (x_2(t) - x_1(t)) $$
$$ y'(t) = y_1(t) + s (y_2(t) - y_1(t)) $$

Este método requiere resolver dos veces el sistema usando un método como Runge-Kutta.

In [1]:
# Definición del main para el problema de contorno por Disparo Lineal
---
En esta sección se implementa la resolución del problema de contorno mediante el método del disparo lineal.

Se definen los siguientes elementos:
- La ecuación diferencial de segundo orden, en formato compatible con `EDOsuperior`.
- Las condiciones de frontera: $y(a) = \alpha$ y $y(b) = \beta$.
- El intervalo $[a, b]$ y el paso de integración $h$.

Se construye una instancia de la clase `ShootLineal`, que internamente:
- Utiliza la clase `EDOsuperior` para convertir la EDO de segundo orden en un sistema de primer orden.
- Aplica dos disparos usando `y'(a) = 0` y `y'(a) = 1`.
- Calcula la combinación lineal de ambas soluciones para cumplir con la condición final $y(b) = \beta$.

Finalmente, el resultado se presenta en forma tabular, incluyendo los valores de $t$, $y(t)$ y $y'(t)$.

# Definición del método del Disparo Lineal
---
blabla

In [7]:
class RungeKuttaSystem:
    '''
    Implementa el método de Runge-Kutta de orden 4 para resolver sistemas de EDOs de primer orden
    de la forma:

            dx/dt = f(t, x, y)
            dy/dt = g(t, x, y)

    Requiere funciones f y g como strings, así como condiciones iniciales para x y y.
    '''
    def __init__(self,function1:str, function2:str, interval:tuple, h_steps:float,x_0:float, y_0:float ):
        """
        Inicializa el sistema con las funciones diferenciales y condiciones iniciales.

        Args:
            function1 (str): Expresión de dx/dt = f(t, x, y).
            function2 (str): Expresión de dy/dt = g(t, x, y).
            interval (tuple): Intervalo de integración (a, b).
            h_steps (float): Paso de integración h.
            x_0 (float): Valor inicial de x en t = a.
            y_0 (float): Valor inicial de y en t = a.
        """
        self.expr1=function1
        self.expr2=function2

        t,x,y = sp.symbols('t x y')
        self.diff_eval_function1=sp.lambdify((t, x, y), self.expr1)
        self.diff_eval_function2=sp.lambdify((t, x, y), self.expr2)

        self.x_0=x_0
        self.y_0=y_0

        self.h=h_steps
        self.a,self.b=interval

        self.M_subintervals=int((self.b-self.a)/self.h)


    def set_h(self,h):
        """
        Permite actualizar el paso de integración h y recalcula el número de subintervalos.

        Args:
            h (float): Nuevo paso de integración.
        """
        self.h=h
        self.M_subintervals=int((self.b-self.a)/self.h)

    def solve_by_rk4(self):
        """
        Resuelve el sistema de EDOs mediante el método de Runge-Kutta de orden 4 (RK4).

        Returns:
            dict: Un diccionario con las listas:
                - "k": índices de iteración
                - "t_k": valores de t
                - "x_k": aproximaciones de x(t)
                - "y_k": aproximaciones de y(t)
        """
        function1=self.diff_eval_function1
        function2=self.diff_eval_function2

        h=self.h

        x_k=self.x_0
        y_k=self.y_0

        t_k_list=[(self.a + k*self.h) for k in range(self.M_subintervals+1)]
        k_iterations=list(range(len(t_k_list)))

        results={"k":k_iterations, "t_k":t_k_list, "x_k":[], "y_k":[]}

        for t_k in t_k_list:

            # aplicacion iterativa de las formulas para sistemas acoplados
            f_1, g_1 = (function1(t_k, x_k, y_k),                           function2(t_k, x_k, y_k))
            f_2, g_2 = (function1(t_k+h/2, x_k+(h/2)*f_1, y_k+(h/2)*g_1),   function2(t_k+h/2, x_k+(h/2)*f_1, y_k+(h/2)*g_1))
            f_3, g_3 = (function1(t_k+h/2, x_k+(h/2)*f_2, y_k+(h/2)*g_2),   function2(t_k+h/2, x_k+(h/2)*f_2, y_k+(h/2)*g_2))
            f_4, g_4 = (function1(t_k+h, x_k+h*f_3, y_k+h*g_3),             function2(t_k+h, x_k+h*f_3, y_k+h*g_3))

            x_k1 = x_k + (f_1+2*f_2+2*f_3+f_4)*h/6
            y_k1 = y_k + (g_1+2*g_2+2*g_3+g_4)*h/6

            results["x_k"].append(x_k)
            results["y_k"].append(y_k)

            x_k=x_k1
            y_k=y_k1                   #actualiza el valor de x_k, y_k

        return results


class EDOsuperior:
    '''
    Recibe una ecuacion mx''(t) + cx'(t) + kx'(t) - g(t) = 0
    y despeja x''(t).

    La entrada de la ecuación debe seguir un formato.

    ddx = x''(t)
    dx = x'(t)
    x = x(t)

    Por lo tanto, la entrada en string para una ecuación diferencial igualada a sero es:
    input>> m*ddx + c*dx + k*x - g
    Nota: m, c, k, g; pueden ser constantes o funciones de t.

    '''

    def __init__(self, ecuation:str):
        """
        Inicializa la ecuación diferencial de segundo orden.

        Args:
            ecuation (str): Ecuación en notación simbólica con 'ddx', 'dx' y 'x'.
        """
        self.ecuation=ecuation

    def _clear_second_derivate(self):
        """
        Despeja la segunda derivada (ddx) de la ecuación.

        Returns:
            str: Expresión de ddx despejada, como string.
        """
        ecuation=self.ecuation
        ddx = sp.Symbol("ddx")

        solution_second_derivate=str(sp.solve(ecuation, ddx)[0])
        return solution_second_derivate

    def _do_substitution(self):
        """
        Sustituye dx por y en la expresión despejada de ddx, para reducir el orden de la EDO.

        Returns:
            str: Ecuación reducida con dx → y.
        """

        substitute_ecuation=self._clear_second_derivate().replace("dx","y")
        return substitute_ecuation

    def solve(self,interval:tuple, h_steps:float, x_0:float, y_0:float):

        ecuation_replaced=self._do_substitution()
        system=RungeKuttaSystem("y", ecuation_replaced, interval, h_steps, x_0, y_0)
        results=system.solve_by_rk4()
        return results



class disparo_lineal:
  #FALTA ESTA CLASE
  pass


# Definición del main para el problema de contorno por Disparo Lineal
---
blablabla

In [9]:
def user_input():
  pass

def main():
  pass


Se ejecuta el main

In [10]:
main()

# Definición del método de Diferencias Finitas
---

# Definición del main para el problema de contorno por Diferencias Finitas
---
blablabla